In [3]:
import faust

In [4]:
from models import KafkaConfig, Topic

config = KafkaConfig.create_from_toml("config.toml")

topic = Topic(config=config)
topic.topic = "hello-topic"
topic.create()

In [13]:
broker = "kafka://" + config.bootstrap_servers
broker_credentials = faust.SASLCredentials(
    username=config.sasl_username,
    password=config.sasl_password,
)

In [14]:
class Greeting(faust.Record):
    from_name: str
    to_name: str

app = faust.App('hello-app', broker=broker, broker_credentials=broker_credentials)

In [10]:
@app.agent(topic)
async def hello(greetings):
    async for greeting in greetings:
        print(f'Hello from {greeting.from_name} to {greeting.to_name}')

@app.timer(interval=1.0)
async def example_sender(app):
    await hello.send(
        value=Greeting(from_name='Faust', to_name='you'),
    )

ValueError: Component 'pkc-lq8gm' of 'pkc-lq8gm.westeurope.azure.confluent.cloud' is not a valid identifier